<a href="https://colab.research.google.com/github/JayThibs/effective-ml-mini-projects/blob/main/hyperparameter_sweeps_using_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to do a Hyperparameter Sweep in Weights and Biases

A hyperparameter sweep is used when we want to do hyperparameter tuning of our machine learning model. Weights and Biases allows uses to do sweeps fairly easily and creates some beautiful graphs to see how well our model did after training it with different sets of hyperparameters.

# An Overview of Sweeps

There a 3 steps to running a sweep with Weights and Biases:

1. **Define the sweep:** create a dictionary or YAML file that specifies the parameters to search through, the search strategy, the optimization strategy, etc.

2. **Initialize the sweep:** initialize the sweep and pass the dictionary of sweep configurations with: `sweep_id = wandb.sweep(sweep_config)`.

3. **Run the sweep agent:** call `wandb.agent(sweep_id, function=train)`, where `function` defines the model architecture and trains it.

If you decide the create a YAML file for your sweep, it should look something like this for a **deep learning** model:

    # sweep.yaml
    program: train.py
    method: random
    metric:
     name: val_loss
     goal: minimize
    parameters:
     learning-rate:
       min: 0.00001
       max: 0.1
     optimizer:
       values: ["adam", "sgd"]
     hidden_layer_size:
       values: [96, 128, 148]
     epochs:
       value: 27
    early_terminate:
       type: hyperband
       s: 2
       eta: 3
       max_iter: 27

And if you use a python dictionary, it should look like this for an xgboost model:

    sweep_config = {
        "method": "random", # try grid or random
        "metric": {
          "name": "accuracy",
          "goal": "maximize"   
        },
        "parameters": {
            "booster": {
                "values": ["gbtree","gblinear"]
            },
            "max_depth": {
                "values": [3, 6, 9, 12]
            },
            "learning_rate": {
                "values": [0.1, 0.05, 0.2]
            },
            "subsample": {
                "values": [1, 0.5, 0.3]
            }
        }
    }

If you want to run a sweep from the command-line, you can run the following commands:

1. **Setup a new sweep:** `wandb sweep sweep.yaml` which creates your sweep, and returns both a unique identifier (SWEEP_ID) and a URL to track all your runs.

2. **Launch the sweep:** `wandb agent SWEEP_ID`, this will start the hyperparameter sweep and return the URL where you can track the sweep's progress. You can also launch multiple agents (GPUs / CPUs) concurrently. Each of these agents will fetch parameters from the W&B server and use them to train the next model.

Documentation on sweeps can be found here: https://docs.wandb.ai/guides/sweeps/quickstart

**Tips:** 

1. You are probably going to end up doing more than one hyperparameter sweep, so start out broad and then hone in on the hyperparameter space with the best performance for your next sweeps.

2. Try to use log ditributed sweeps (especially for batch size, learning rate, and hidden layer size). Instead of doing sweeps uniformly between every value in 1 to 1000, you can try every order of magnitude (1, 10, 100, 1000). For example, `q_log_uniform` will try different orders of magnitude with equal probability.

Now, let's get started with code!

## Setup





In [1]:
!pip install wandb --upgrade

import wandb

wandb.login()

     |████████████████████████████████| 1.6 MB 13.6 MB/s 
     |████████████████████████████████| 97 kB 6.4 MB/s 
     |████████████████████████████████| 170 kB 71.5 MB/s 
     |████████████████████████████████| 133 kB 53.2 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=f7f120447558a9ca8a7cdf421ff96b883521b21a0864e8ec318dd7fe6c261f2c
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=c193af086847df8df831f03589fd14f36f2a6afbf9870fa9236c16b70b5daa82
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Step 1. Define the Sweep

Fundamentally, a Sweep combines a strategy for trying out a bunch of hyperparameter values with the code that evaluates them. We configure it with different hyperparameters and a method like bayesian optimization (`bayes`) or `random` search.

Usually we choose either random search or bayesian search. Bayesian is great, but it does not do so well when you have a ton of hyperparameters (scales poorly as a function of number of hyperparameters).

In [7]:
import math

sweep_config = {
    'method': 'random',
    'metric': {
    'name': 'loss',
    'goal': 'minimize'
    },
    'parameters': {
    'optimizer': {
        'values': ['adam', 'sgd']
    },
    'fc_layer_size': {
        'values': [128, 256, 512]
    },
    'dropout': {
        'values': [0.3, 0.4, 0.5]
    },
    'epochs': {
        'value': 1
    },
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
    },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms
        'distribution': 'q_log_uniform', # Quantized log uniform. Returns round(X, q) 
                                         # allows you to try every order of magnitude uniformly
        'q': 1,
        'min': math.log(32),
        'max': math.log(256),
    }
  }
}

# We can also include hyperband for early stopping

In [8]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform',
                               'max': 5.545177444479562,
                               'min': 3.4657359027997265,
                               'q': 1},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'value': 1},
                'fc_layer_size': {'values': [128, 256, 512]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam', 'sgd']}}}


# Step 2. Initialize the Sweep

Weights and Biases has something called the Sweep Controller that handles the Sweep and issues a new set of instructions describing a new run to execute locally on our machines.

In [9]:
sweep_id = wandb.sweep(sweep_config, project='pytorch-sweeps-demo')

Create sweep with ID: 5rmgndiy
Sweep URL: https://wandb.ai/jacquesthibs/pytorch-sweeps-demo/sweeps/5rmgndiy
